In [14]:
# Workspace problem with several narrow gaps

import torch
import torch.nn as nn

from functools import partial
import numpy as np
from numpy.random import default_rng
from io import StringIO
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import matplotlib.animation as animation
import matplotlib.patches as patches
import sys
import time
import cv2
from celluloid import Camera
import time
from IPython.display import HTML

sys.path.append('/home/oscar_palfelt/MSc_thesis/ompl/py-bindings')
from ompl import base as ob
from ompl import control as oc
from ompl import geometric as og

rng = default_rng(900)

cuda = False
DEVICE = torch.device("cuda" if cuda else "cpu")

In [16]:
# neural network parameters
mb_size = 256 # mini batch dim
h_Q_dim = 512*1 # encoder dim
h_P_dim = 512*1 # decoder dim

# problem dimenc_dimsions
nDrawnSamples = 21 # number of dependent samples to draw during smapling (length of predicted control series)
nPreviousStates = 1 # number of previous states to condition on the steering prediction
dim = 7 # (truckX, truckY, truckYaw, truckX, trailerY, trailerYaw, steer)
dimGoal = 3
dimObs = 2
dimPredict = dim - 0 # predict truck (truckX, truckY, truckYaw, steer)
dataElements = (nDrawnSamples + nPreviousStates) * dim + dimGoal + dimObs # steer sample, current pose (3D: x,y,yaw), goal (3D), dimObs (2D x 3)

z_dim = 3 # latent dim
X_dim = nDrawnSamples*dimPredict
c_dim = dataElements - X_dim # dimension of conditioning variable

In [6]:
# define pytorch networks
# based on https://github.com/Jackson-Kang/Pytorch-VAE-tutorial/blob/master/.ipynb_checkpoints/01_Variational_AutoEncoder-checkpoint.ipynb

class Encoder(nn.Module):
    
    def __init__(self, input_dim=X_dim+c_dim, hidden_dim=h_Q_dim, latent_dim=z_dim):
        super(Encoder, self).__init__()

        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU()
        )

        self.z_mu = nn.Linear(hidden_dim, latent_dim)
        self.z_logvar = nn.Linear(hidden_dim, latent_dim)
        
    def forward(self, x):

        seq = self.network(x)

        return self.z_mu(seq), self.z_logvar(seq)


class Decoder(nn.Module):
    def __init__(self, latent_dim=z_dim+c_dim, hidden_dim=h_P_dim, output_dim=X_dim):
        super(Decoder, self).__init__()

        self.network = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
        
    def forward(self, x):
        
        return self.network(x)


class NeuralNetwork(nn.Module):
    def __init__(self, Encoder, Decoder):
        super(NeuralNetwork, self).__init__()
        self.Encoder = Encoder
        self.Decoder = Decoder
    
    def reparameterization(self, mean, logvar):
        epsilon = torch.randn_like(mean).to(DEVICE)        # sampling epsilon        
        z = mean + torch.exp(0.5 * logvar) * epsilon       # reparameterization trick
        return z

    def forward(self, x, c, encode=True):
        if encode:
            z_mu, z_logvar = self.Encoder(torch.cat((x, c), dim=1))
            z = self.reparameterization(z_mu, z_logvar)

            y = self.Decoder(torch.cat((z, c), dim=1))
            
            return y, z_mu, z_logvar
        else:
            z = x
            y = self.Decoder(torch.cat((z, c), dim=1))    

            return y

network = torch.load('/home/oscar_palfelt/MSc_thesis/EECS_Degree_Project/learn_control/truck_trailer/networks/intersect_23.pt', map_location=torch.device(DEVICE))

In [17]:
# read in data from .txt file, re-arrange to allow drawing multiple depedent samples

filename = '/home/oscar_palfelt/MSc_thesis/EECS_Degree_Project/learn_control/truck_trailer/motion_planning/data/ctrlTrailerIntersection_10kplans'
rawdata = np.genfromtxt(filename, delimiter=',', dtype='d', usecols=[0,1,2,3,4,5,6,10,11,12,13,14]) # disregard init

_, pathsIdx = np.unique(rawdata[:,dim:], axis=0, return_index=True)
pathsIdx.sort()

pathsLengths = np.roll(pathsIdx, -1) - pathsIdx
pathsLengths[-1] = rawdata.shape[0] - np.sum(pathsLengths[:-1])

validLengthsIdx = np.argwhere(pathsLengths >= nPreviousStates + nDrawnSamples) 
validPlansIdx = pathsIdx[validLengthsIdx]

data = np.zeros(shape=(1, dataElements)) # steering angles, samples (x,y,yaw), goal(x,y,yaw)
tempdata = np.copy(data)
for ci, i in enumerate(validPlansIdx[:-1].reshape(-1)):
    for j in range(pathsLengths[ci] - (nPreviousStates + nDrawnSamples - 1)):
        sample = np.arange(start=j, stop=j + nPreviousStates + nDrawnSamples)

        if np.any(np.abs(np.diff(rawdata[i + sample, 0])) > 0.05) or np.any(np.abs(np.diff(rawdata[i + sample, 1])) > 0.05):
            break # ignore duplicate plans (data generation is not perfect)
        
        seqstates = rawdata[i + sample, :dim].reshape(1, (nPreviousStates + nDrawnSamples)*dim)
        if np.where(np.abs(seqstates[0, 4:(nPreviousStates + nDrawnSamples)*dim:dim]) < 3e-2)[0].shape[0] >= (nDrawnSamples + nPreviousStates) // 1.2:
            continue

        tempdata = np.vstack((tempdata, np.append(seqstates, rawdata[i, dim:].reshape(1, dimGoal+dimObs), axis=1))) 

        if ci % 500 == 0:
            data = np.vstack((data, tempdata[1:]))
            tempdata = np.zeros(shape=(1, dataElements))

data = data[1:]

rng.shuffle(data) # shuffle data (otherwise the data is sorted by obstacles/the map)

pathData = np.copy(data[:, :-dimObs])
obsData = np.copy(data[:, -dimObs:])

numEntries = pathData.shape[0]
print(numEntries)

31505


In [8]:
ratioTestTrain = 0.8;
numTrain = int(numEntries*ratioTestTrain)

nextStates = pathData[:, nPreviousStates*dim:-dimGoal]

prevStatesConditions = pathData[:, :nPreviousStates * dim]
goalStateCondition = pathData[:, -dimGoal:]

conditionsOcc = np.copy(obsData)

cs = np.concatenate((prevStatesConditions, goalStateCondition, conditionsOcc), axis=1)
c_train = cs[0:numTrain,:]  
c_test = cs[numTrain:numEntries,:]

In [9]:
def dist2linesegment(p, a, b): # p: point, a: line end 1, b: line ned 2
    """
    # from https://stackoverflow.com/a/58781995

    Args:
        - p: np.array of single point, shape (2,) or 2D array, shape (x, 2)
        - a: np.array of shape (x, 2)
        - b: np.array of shape (x, 2)
    """

    # normalized tangent vectors
    d_ba = b - a
    d = np.divide(d_ba, (np.hypot(d_ba[:, 0], d_ba[:, 1])
                           .reshape(-1, 1)))

    # signed parallel distance components
    # rowwise dot products of 2D vectors
    s = np.multiply(a - p, d).sum(axis=1)
    t = np.multiply(p - b, d).sum(axis=1)

    # clamped parallel distance
    h = np.maximum.reduce([s, t, np.zeros(len(s))])

    # perpendicular distance component
    # rowwise cross products of 2D vectors  
    d_pa = p - a
    c = d_pa[:, 0] * d[:, 1] - d_pa[:, 1] * d[:, 0]

    return np.hypot(h, c)

In [13]:
# define planning problem
class MyGoal(ob.Goal):
    def __init__(self, si, goal):
        super(MyGoal, self).__init__(si)
        self.goal = goal

    def isSatisfied(self, state):
        return np.linalg.norm(self.goal - np.array([state[0], state[1], (state[2] + np.pi) % (2 * np.pi) - np.pi])) < 0.03


def getThresholdPathLengthObj(si):
     obj = ob.PathLengthOptimizationObjective(si)
     obj.setCostThreshold(ob.Cost(4.0))
     return obj
     

def isRV4Valid(spaceInformation, state):
    # perform collision checking or check if other constraints are
    # satisfied

    pTruck = np.array([state[0], state[1]])

    xTrailer = state[0] - b1 * np.cos(state[2]) - np.linspace(0, b2, 6) * np.cos(state[3])
    yTrailer = state[1] - b1 * np.sin(state[2]) - np.linspace(0, b2, 6) * np.sin(state[3])
    pTrailer = np.array([xTrailer, yTrailer]).T

    l1distTruck = dist2linesegment(pTruck, a=l1a, b=l2a)
    l2distTruck = dist2linesegment(pTruck, a=l1b, b=l2b)

    l1distTrailer = dist2linesegment(pTrailer, a=np.tile(l1a, reps=(pTrailer.shape[0], 1)), b=np.tile(l2a, reps=(pTrailer.shape[0], 1)))
    l2distTrailer = dist2linesegment(pTrailer, a=np.tile(l1b, reps=(pTrailer.shape[0], 1)), b=np.tile(l2b, reps=(pTrailer.shape[0], 1)))

    if spaceInformation.satisfiesBounds(state):
        return np.where((l1distTruck <= roadWidth / 2) | (l2distTruck <= roadWidth / 2))[0].shape[0] == 1 and np.where((l1distTrailer <= roadWidth / 2) | (l2distTrailer <= roadWidth / 2))[0].shape[0] == pTrailer.shape[0]


def propagate(start, control, duration, state):
    global nExploredStates
    nExploredStates += 1

    beta = np.arctan(np.tan(control[0]) * a12 / (a11 + a12))

    state[0] = start[0] + v * np.cos(start[2] + beta) * duration
    state[1] = start[1] + v * np.sin(start[2] + beta) * duration
    state[2] = start[2] + v * np.sin(beta) / a12 * duration
    state[3] = start[3] + (v * np.sin(start[2] - start[3]) / (b2 + a21) - v * (b1 - a12) * np.cos(start[2] - start[3]) * np.sin(beta) / (a12 * (b2 + a21))) * duration


def problemDef():

    # construct the state space we are planning in
    space = ob.RealVectorStateSpace(4)

    # # set the bounds for the R^2 part of SE(2)
    bounds = ob.RealVectorBounds(4)
    bounds.setLow(0, 0.001)
    bounds.setLow(1, 0.001)
    bounds.setLow(2, -2 * np.pi)
    bounds.setLow(3, -2 * np.pi)
    bounds.setHigh(0, 0.999)
    bounds.setHigh(1, 0.999)
    bounds.setHigh(2, 2 * np.pi)
    bounds.setHigh(3, 2 * np.pi)
    space.setBounds(bounds)

    # create a control space
    cspace = oc.RealVectorControlSpace(space, 1)

    # set the bounds for the control space
    cbounds = ob.RealVectorBounds(1)
    cbounds.setLow(-0.65)
    cbounds.setHigh(0.65)
    cspace.setBounds(cbounds)

    ss = oc.SimpleSetup(cspace)
    ss.setStatePropagator(oc.StatePropagatorFn(propagate))
    si = ss.getSpaceInformation()

    si.setPropagationStepSize(.07)
    si.setMinMaxControlDuration(1, 4)
    planner = oc.RRT(si)

    ss.setStateValidityChecker(ob.StateValidityCheckerFn( \
        partial(isRV4Valid, ss.getSpaceInformation())))

    ss.setPlanner(planner)
    ss.getProblemDefinition().setOptimizationObjective(getThresholdPathLengthObj(si))
    
    return ss


def plan(planObj, initState, goalState):

    space = ob.RealVectorStateSpace(4)

    start = ob.State(space)
    start[0] = initState[0] # truck x
    start[1] = initState[1] # truck y
    start[2] = initState[2] # truck yaw
    start[3] = initState[3] # trailer yaw

    si = planObj.getSpaceInformation()
    planObj.setStartState(start)
    planObj.setGoal(MyGoal(si, goalState))

    timeTerminationCondition = ob.timedPlannerTerminationCondition(20.)

    costTerminationCondition = ob.CostConvergenceTerminationCondition(planObj.getProblemDefinition(), solutionsWindow=4, epsilon=1.0)

    solved = planObj.solve(ob.plannerOrTerminationCondition(costTerminationCondition, timeTerminationCondition))


In [11]:
class VehicleState():
    def __init__(self):
        self.x = 0
        self.y = 0
        self.yaw = 0
        self.v = 0

In [14]:
# plot the latent space
num_viz = 800 # number of samples to draw in latent space

dt = 0.2 # controller time step
v =  0.1 # vehicle speed
L   = 0.05 # truck length constant
W   = 0.025 # truck/trailer width
Lt  = 0.20 # trailer length constant
a11 = 0.5 * L
a12 = L - a11
b1  = 0.9 * a12
b2  = 0.5 * Lt 
a21 = Lt - b2

roadWidth = 0.12
c = np.array([0.5, 0.5]) # road center

pdef = problemDef()
t = np.tile(np.arange(nDrawnSamples), reps=(num_viz, 1)) * dt

nTests = 1000
nCompletedTests = 0
nAttemptedTests = 0
animate = False
CVAEtime = np.zeros(shape=nTests)
RRTtime = np.zeros(shape=nTests)
nCVAEstates = np.zeros(shape=nTests)
nRRTstates = np.zeros(shape=nTests)
nCVAEsuccess = 0
nRRTsuccess = 0
nCVAEfails = 0
nRRTfails = 0

while nCompletedTests < nTests:
    nAttemptedTests += 1

    collision = False

    vizIdx = np.random.randint(numTrain,numEntries-1) # chose a random test scenario

    start = pathData[vizIdx, :dim]
    goal = data[vizIdx, -dimGoal-dimObs:-dimObs]
    alpha1 = obsData[vizIdx, 0]
    alpha2 = obsData[vizIdx, 1]
    a = c[0] + 0.5 * np.array([np.cos(alpha1), np.sin(alpha1)]) # line segment 1 end point
    b = c[1] + 0.5 * np.array([np.cos(alpha2), np.sin(alpha2)]) # line segment 2 end point
    l1a = a.reshape(1,2)
    l2a = c.reshape(1,2)
    l1b = b.reshape(1,2)
    l2b = np.copy(l2a)
    sign = np.where(alpha1 < alpha2, 1, -1)
    k = (1 / np.abs(np.sin(.5*(alpha1 - alpha2 + np.pi + np.pi) % (2 * np.pi) - np.pi)))
    lcx = c[0] + sign * roadWidth / 2 * np.cos((alpha1 + alpha2)/2) * k
    lcy = c[1] + sign * roadWidth / 2 * np.sin((alpha1 + alpha2)/2) * k
    scx = c[0] - sign * roadWidth / 2 * np.cos((alpha1 + alpha2)/2) * k
    scy = c[1] - sign * roadWidth / 2 * np.sin((alpha1 + alpha2)/2) * k

    truckState = VehicleState()
    truckState.x = start[0]; truckState.y = start[1]; truckState.yaw = start[2]; truckState.v = v

    trailerState = VehicleState()
    trailerState.yaw = start[3]
    trailerState.x = start[5]
    trailerState.y = start[6]

    steerPrev = prevStatesConditions[vizIdx, 4::dim]
    delta = steerPrev[-1]

    maxSteer = np.max(pathData[:, 4::dim])
    minSteer = np.min(pathData[:, 4::dim])
    steerAngles = np.linspace(minSteer, maxSteer, num=200)

    CVAEstartTime = time.time()
    c_sample_seed = cs[vizIdx,:]
    c_sample = torch.from_numpy(np.repeat([c_sample_seed],num_viz,axis=0)).float().to(DEVICE)
    y_viz = network(torch.randn(num_viz, z_dim).to(DEVICE), c_sample, encode=False).cpu().detach().numpy()

    kdes = np.array([gaussian_kde(y_viz[:, i * dim - 4]) for i in range(1, nDrawnSamples + 1)])

    if animate:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,8), gridspec_kw={'width_ratios': [3, 2]})
        camera = Camera(fig)

    for kde in kdes:

        delta = steerAngles[np.argmax(kde.pdf(steerAngles))]

        beta = np.arctan(np.tan(delta) * a12 / (a11 + a12))

        trailerState.x = truckState.x - b1 * np.cos(truckState.yaw) - b2 * np.cos(trailerState.yaw)
        trailerState.y = truckState.y - b1 * np.sin(truckState.yaw) - b2 * np.sin(trailerState.yaw)
        trailerState.yaw += (v * np.sin(truckState.yaw - trailerState.yaw) / (b2 + a21) - v * (b1 - a12) * np.cos(truckState.yaw - trailerState.yaw) * np.sin(beta) / (a12 * (b2 + a21))) * dt

        truckState.x += v * np.cos(truckState.yaw + beta) * dt
        truckState.y += v * np.sin(truckState.yaw + beta) * dt
        truckState.yaw += v * np.sin(beta) / a12 * dt

        # ------------ collision checking ------------
        pTruck = np.array([truckState.x, truckState.y])

        xTrailer = truckState.x - b1 * np.cos(truckState.yaw) - np.linspace(0, b2, 6) * np.cos(trailerState.yaw)
        yTrailer = truckState.y - b1 * np.sin(truckState.yaw) - np.linspace(0, b2, 6) * np.sin(trailerState.yaw)
        pTrailer = np.array([xTrailer, yTrailer]).T

        l1distTruck = dist2linesegment(pTruck, a=l1a, b=l2a)
        l2distTruck = dist2linesegment(pTruck, a=l1b, b=l2b)

        l1distTrailer = dist2linesegment(pTrailer, a=np.tile(l1a, reps=(pTrailer.shape[0], 1)), b=np.tile(l2a, reps=(pTrailer.shape[0], 1)))
        l2distTrailer = dist2linesegment(pTrailer, a=np.tile(l1b, reps=(pTrailer.shape[0], 1)), b=np.tile(l2b, reps=(pTrailer.shape[0], 1)))

        if truckState.x >= 0. and truckState.x <= 1. and truckState.y >= 0. and truckState.y <= 1.:
            if not np.where((l1distTruck <= roadWidth / 2) | (l2distTruck <= roadWidth / 2))[0].shape[0] == 1 and np.where((l1distTrailer <= roadWidth / 2) | (l2distTrailer <= roadWidth / 2))[0].shape[0] == pTrailer.shape[0]:
                collision = True
                break
            collision = False
        else:
            collision = True
            break
        # --------------------------------------------
     

        if animate:
            ax1.scatter(y_viz[:,::dim], y_viz[:,1::dim], color='green', s=70, alpha=0.006, label='CVAE predicted (x,y) states') # nn samples
            ax1.quiver(goal[0], goal[1], np.cos(goal[2]), np.sin(goal[2]), color="blue", scale=11.0, width=0.012, label='Goal pose') # goal
            ax1.plot([a[0] + roadWidth / 2 * np.sin(alpha1), scx], [a[1] - roadWidth / 2 * np.cos(alpha1), scy], color='black')
            ax1.plot([a[0] - roadWidth / 2 * np.sin(alpha1), lcx], [a[1] + roadWidth / 2 * np.cos(alpha1), lcy], color='black')
            ax1.plot([b[0] + roadWidth / 2 * np.sin(alpha2), lcx], [b[1] - roadWidth / 2 * np.cos(alpha2), lcy], color='black')
            ax1.plot([b[0] - roadWidth / 2 * np.sin(alpha2), scx], [b[1] + roadWidth / 2 * np.cos(alpha2), scy], color='black')
            rectTruck = patches.Rectangle((truckState.x - np.cos(truckState.yaw) * L/2 + np.sin(truckState.yaw) * W/2, 
                        truckState.y - np.cos(truckState.yaw) * W/2 - np.sin(truckState.yaw) * L/2), L, W, angle=np.rad2deg(truckState.yaw), color='purple', alpha=0.8, label='Vehicle')
            rectTrailer = patches.Rectangle((truckState.x - b1 * np.cos(truckState.yaw) - b2 / 2 * np.cos(trailerState.yaw) - np.cos(trailerState.yaw) * b2/2 + np.sin(trailerState.yaw) * W/2, 
                        truckState.y - b1 * np.sin(truckState.yaw) - b2 / 2 * np.sin(trailerState.yaw) - np.cos(trailerState.yaw) * W/2 - np.sin(trailerState.yaw) * b2/2), b2, W, angle=np.rad2deg(trailerState.yaw), color='orangered', alpha=0.8, label='Trailer')
            ax1.add_patch(rectTruck)
            ax1.add_patch(rectTrailer)
            ax1.set_xlabel('x', fontsize=12)
            ax1.set_ylabel('y', fontsize=12)
            ax1.set_xlim(0, 1)
            ax1.set_ylim(0, 1)
            ax2.plot(steerAngles, kde.pdf(steerAngles), color='C0')
            ax2.set_xlabel('Steering angle', fontsize=12)
            ax2.set_ylabel('Sample density', fontsize=12)
            plt.show()

            camera.snap()

    CVAEendTime = time.time()
    
    if not collision:
        nCVAEsuccess += 1

        pdef.clear()
        nExploredStates = 0
        RRTstartTime = time.time()
        plan(pdef, initState=start[:4], goalState=[truckState.x, truckState.y, truckState.yaw])
        if pdef.getProblemDefinition().hasExactSolution():
            RRTendTime = time.time()
            RRTtime[nCompletedTests] = RRTendTime - RRTstartTime
            nRRTstates[nCompletedTests] = nExploredStates

            CVAEtime[nCompletedTests] = CVAEendTime - CVAEstartTime
            nCVAEstates[nCompletedTests] = nDrawnSamples # constant

            nRRTsuccess += 1
            nCompletedTests += 1
        else:
            nRRTfails += 1

        if animate:
            fps = 15
            animation = camera.animate(interval=1000/fps)
            print("saving animation")
            animation.save('animation.mp4')

    else:    
        nCVAEfails += 1

Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 3989 states
Info:    Solution found in 5.406689 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    ProblemDefinition: Adding approximate solution from planner RRT
Info:    RRT: Created 17556 states
Info:    Solution found in 20.000264 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 400 states
Info:    Solution found in 0.659061 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 10261 states
Info:    Solution found in 11.219412 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 15685 states
Info:    Solution found in 16.822865 seconds
Info:    RRT: Starting planning with 1 states already in datastructure
Info:    RRT: Created 671 states
Info:    Solution found in 1.107783 seconds
Info:    RRT: Starting planni

KeyboardInterrupt: 

In [16]:
print('Mean CVAE CPU time: {}, std: {}'.format(np.mean(CVAEtime[:nCompletedTests]), np.std(CVAEtime[:nCompletedTests])))
print('Mean RRT CPU time: {}, std: {}'.format(np.mean(RRTtime[:nCompletedTests]), np.std(RRTtime[:nCompletedTests])))

print('Mean CVAE #states: {}, std: {}'.format(np.mean(nCVAEstates[:nCompletedTests]), np.std(nCVAEstates[:nCompletedTests])))
print('Mean RRT #states: {}, std: {}'.format(np.mean(nRRTstates[:nCompletedTests]), np.std(nRRTstates[:nCompletedTests])))

print('CVAE success rate: {}%'.format(nCVAEsuccess / (nCVAEsuccess + nCVAEfails) * 100))
print('RRT success rate: {}%'.format(nRRTsuccess / (nRRTsuccess + nRRTfails) * 100))

Mean CVAE CPU time: 0.06796022181241017, std: 0.019987534963759303
Mean RRT CPU time: 5.840099631615405, std: 5.550713978661094
Mean CVAE #states: 23.0, std: 0.0
Mean RRT #states: 27148.943396226416, std: 25169.311684964872
CVAE success rate: 88.79310344827587%
RRT success rate: 51.9607843137255%
